In [1]:
import requests
import pandas as pd
import json
import time
import pyodbc 
import urllib
import sqlalchemy
import os
import bs4
import html5lib

In [7]:
##cria conexão com o banco

DRIVER = 'SQL SERVER'
SERVER = 'DESKTOP-2VSV2T8'
DB_NAME = 'Projeto_Steam'

conn = pyodbc.connect(f"""
        DRIVER={{{DRIVER}}};
        SERVER={SERVER};
        DATABASE={DB_NAME};
        Trust_Connection=yes;
                        """)

##cria cursor para execução dos comandos em sql
cursor = conn.cursor()

##comita para salvar as alterações
cursor.commit()



In [ ]:
##valida numero de paginas da api com os jogos
steam_spy = f'https://steamspy.com/api.php?request=all&page={63}'
novo_request = requests.get(steam_spy)



if novo_request.status_code == 200:
    print('true')
else:
     print('false')

In [ ]:
##extrai dados da api

pagination = 1
lista_de_jogos = []

# steam_spy = 'https://steamspy.com/api.php?request=all&page=1'
# novo_request = requests.get(steam_spy)
# df = json.loads(novo_request.text)


while True :
    try:
        steam_spy = f'https://steamspy.com/api.php?request=all&page={pagination}'
        novo_request = requests.get(steam_spy)
        df =  json.loads(novo_request.text) 
        for x in df:
            lista_de_jogos.append((df[x].get('appid'), df[x].get('name'),df[x].get('developer'),df[x].get('publisher'),df[x].get('ccu'),df[x].get('positive'),df[x].get('negative'),df[x].get('languages')))
        
        pagination = pagination + 1
    except:
        print('carga finalizada')
        break







In [ ]:
##tranforma em df e renomeia colunas
df = pd.DataFrame(lista_de_jogos)
df.columns = ['appid','nome','desenvolvedor','distribuidora','ccu','positivo','negativo', 'idiomas']

df


In [ ]:
##cria tabela de aplicativos resumida
cursor.execute('DROP TABLE IF EXISTS Jogos CREATE TABLE Jogos (Appid bigint, Nome varchar(1000))')

##insere dados dos jogos na tabela
for index,row in df.iterrows():
    cursor.execute(f'insert into Jogos  (appid,nome) values(?,?)',row.appid,row.nome)
    


##cria tabela fact jogos
cursor.execute('DROP TABLE IF EXISTS Fact_Jogos CREATE TABLE Fact_Jogos (Appid bigint, Nome varchar(1000),desenvolvedor varchar(1000),distribuidora varchar(1000),ccu bigint, positivo bigint, negativo bigint)')
cursor.commit()

for index,row in df.iterrows():
    cursor.execute(f'insert into Fact_Jogos  (appid,nome,desenvolvedor,distribuidora,ccu,positivo,negativo) values(?,?,?,?,?,?,?)',row.appid,row.nome,row.desenvolvedor,row.distribuidora,row.ccu,row.positivo,row.negativo)

cursor.commit()


In [ ]:
##le dados de uma tabela
pd.read_sql_query('select * from Fact_Jogos',con=conn) 

#PARTE 2 DADOS DAS PLACAS

In [ ]:
# Faz o download do conteúdo da página
url = "https://store.steampowered.com/hwsurvey/videocard/"
response = requests.get(url)
content = response.content

# Cria um objeto BeautifulSoup para analisar o conteúdo HTML
soup = bs4.BeautifulSoup(content, "html.parser")

data_list = []
for i in range(0,2,1) :
# Encontra todas as divs com a classe "substats_row row_0"
    divs = soup.find_all("div", class_=f"substats_row row_{i}")

    # Extrai os dados de cada div
    for div in divs:
        data = div.get_text(separator='|', strip=True)
        data_list.append(data)

# transforma em dataframe
df = pd.DataFrame(data_list,columns=['A'])
#separa as colunas do dataframe
df = df['A'].str.split('|',expand=True)

#renomeia colunas
df.columns = ['Placa','DEZ','JAN','FEV','MAR','ABR','RESULTADO']

# #identifica quais linhas quero remover
# remover = df.query('Placa.str.contains("Direct")').index

# #remove registros
# df.drop(remover,axis=0,inplace=True)

substituicoes = {'%':'','-':'0'}

# df[['DEZ','JAN','FEV','MAR','ABR']].replace(substituicoes,regex=True).astype(float)/100

df.iloc[0:,1:].replace(substituicoes,regex=True,inplace=True)

df.drop(columns='RESULTADO')

df['RESULTADO'] = df['ABR'].astype(float)  - df['MAR'].astype(float)

df




In [2]:
## Faz o download do conteúdo da página parte 1
url = "https://store.steampowered.com/hwsurvey/videocard/"
response = requests.get(url)
content = response.content

# Cria um objeto BeautifulSoup para analisar o conteúdo HTML
soup = bs4.BeautifulSoup(content, "html.parser")

data_list1 = []
for i in range(0,2,1) :
# Encontra todas as divs com a classe "substats_row row_0"
    divs = soup.find_all("div", class_=f"substats_row row_0")

    # Extrai os dados de cada div
    for div in divs:
        data = div.get_text(separator='|', strip=True)
        data_list1.append(data)

# transforma em dataframe
df = pd.DataFrame(data_list1,columns=['A'])
#separa as colunas do dataframe
df = df['A'].str.split('|',expand=True)

#renomeia colunas
df.columns = ['Placa','MES_1','MES_2','MES_3','MES_4','MES_ATUAL','RESULTADO']

#identifica quais linhas quero remover
remover = df.query('Placa.str.contains("Direct")').index

#remove registros
df.drop(remover,axis=0,inplace=True)

substituicoes = {'%':'','-':'0'}

# df[['DEZ','JAN','FEV','MAR','ABR']].replace(substituicoes,regex=True).astype(float)/100

df.iloc[0:,1:].replace(substituicoes,regex=True,inplace=True)

df.drop(columns='RESULTADO')

df['RESULTADO'] = df['MES_ATUAL'].astype(float)  - df['MES_4'].astype(float)

df1 = df

## Faz o download do conteúdo da página parte 2
url = "https://store.steampowered.com/hwsurvey/videocard/"
response = requests.get(url)
content = response.content

# Cria um objeto BeautifulSoup para analisar o conteúdo HTML
soup = bs4.BeautifulSoup(content, "html.parser")

data_list2 = []
for i in range(0,2,1) :
# Encontra todas as divs com a classe "substats_row row_0"
    divs = soup.find_all("div", class_=f"substats_row row_1")

    # Extrai os dados de cada div
    for div in divs:
        data = div.get_text(separator='|', strip=True)
        data_list2.append(data)

# transforma em dataframe
df = pd.DataFrame(data_list2,columns=['A'])
#separa as colunas do dataframe
df = df['A'].str.split('|',expand=True)

#renomeia colunas
df.columns = ['Placa','MES_1','MES_2','MES_3','MES_4','MES_ATUAL','RESULTADO']

#identifica quais linhas quero remover
remover = df.query('Placa.str.contains("Direct")').index

#remove registros
df.drop(remover,axis=0,inplace=True)

substituicoes = {'%':'','-':'0'}

# df[['DEZ','JAN','FEV','MAR','ABR']].replace(substituicoes,regex=True).astype(float)/100

df.iloc[0:,1:].replace(substituicoes,regex=True,inplace=True)

df.drop(columns='RESULTADO')

df['RESULTADO'] = df['MES_ATUAL'].astype(float)  - df['MES_4'].astype(float)

df2 = df


##combina as duas listas
dados_combinados = [item for pair in zip(data_list1, data_list2) for item in pair]

In [4]:

# transforma em dataframe
df = pd.DataFrame(dados_combinados,columns=['A'])
#separa as colunas do dataframe
df = df['A'].str.split('|',expand=True)

# #renomeia colunas
df.columns = ['Placa','ABR','MAIO','JUN','JUL', 'AUG','RESULTADO']

##remove as 6 primeiras linhas
df = df.drop(index=range(6)).reset_index(drop=True)

# # #identifica quais linhas quero remover
remover = df.query('Placa.str.contains("NVIDIA GeForce GTX 1650")').index



##MARCA A PARTIR DE ONDE QUERO EXCLUIR
ultima_linha = remover[3]
df = df[:ultima_linha]

##CRIA DICIONARIO COM AS SUBSTITUICOES QUE QUERO FAZER NAS COLUNAS DO DATAFRAME
substituicoes = {'%':'','-':'0'}
df.iloc[0:,1:].replace(substituicoes,regex=True,inplace=True)


##CRIA NOVA COLUNA DE RESULTADO FLOAT
df.drop(columns='RESULTADO')
df['RESULTADO'] = df['AUG'].astype(float)  - df['JUL'].astype(float)

##SEPARA POR MARCA
marca = df['Placa'].str.split(' ',n=1,expand=True)[0]
placa = df['Placa'].str.split(' ',n=1,expand=True)[1]
df['Marca'] = marca
df['Placa'] = placa

##corrigi classificação intel
df['Marca'].replace('Intel(R)', 'INTEL',inplace=True)
df['Marca'].replace('Other', 'OUTRAS', inplace=True)

df


,Placa,ABR,MAIO,JUN,JUL,AUG,RESULTADO,Marca
0,GeForce GTX 1650,6.19,6.07,5.67,5.30,5.11,-0.19,NVIDIA
1,GeForce RTX 3060,4.66,4.90,4.74,4.88,4.87,-0.01,NVIDIA
2,GeForce GTX 1060,4.99,4.94,4.58,4.60,4.31,-0.29,NVIDIA
3,GeForce RTX 3060 Laptop GPU,4.51,4.54,4.28,3.76,4.29,0.53,NVIDIA
4,GeForce RTX 2060,4.45,4.53,4.15,4.10,3.94,-0.16,NVIDIA
...,...,...,...,...,...,...,...,...
95,Radeon R7 Graphics,0.17,0.16,0.16,0.17,0.16,-0.01,AMD
96,GeForce GTX 750,0.19,0.17,0.18,0.17,0.16,-0.01,NVIDIA
97,HD Graphics 530,0,0,0.15,0.17,0.16,-0.01,Intel
98,None,9.44,9.48,9.92,10.01,10.06,0.05,OUTRAS


In [ ]:
##cria tabela para as informações das placas

cursor.execute('drop table if exists tb_placas_de_video_resultado  create table Tb_Placas_De_Video_Resultado (Marca varchar(100), placa varchar(200), FEV NUMERIC(20,2),MAR NUMERIC(20,2),ABR NUMERIC(20,2),MAIO NUMERIC(20,2),JUNHO NUMERIC(20,2),RESULTADO NUMERIC(20,2))')
cursor.commit()




In [ ]:
##Insere dados nas tabelas

for index,row in df.iterrows():
    cursor.execute(f'insert into tb_placas_de_video_resultado(Marca,placa,FEV,MAR,ABR,MAIO,JUNHO,RESULTADO) values(?,?,?,?,?,?,?,?)',row[7],row[0],row[1],row[2],row[3],row[4],row[5],row[6])

cursor.commit()

##ATUALIZA MES

In [5]:
##extrai ultimo mês

df = pd.DataFrame(dados_combinados,columns=['A'])

#separa as colunas do dataframe
df = df['A'].str.split('|',expand=True)

df.columns = ['Placa','MES_1','MES_2','MES_3','MES_4','MES_ATUAL','RESULTADO']

##remove as 6 primeiras linhas
df = df.drop(index=range(6)).reset_index(drop=True)

# # #identifica quais linhas quero remover
remover = df.query('Placa.str.contains("NVIDIA GeForce GTX 1650")').index



##MARCA A PARTIR DE ONDE QUERO EXCLUIR
ultima_linha = remover[3]
df = df[:ultima_linha]

##LIMPA CARACETER %
substituicoes = {'%':'','-':'0'}
df.iloc[0:,1:].replace(substituicoes,regex=True,inplace=True)


##corrigi coluna de resultado
df.drop(columns='RESULTADO')
df['RESULTADO'] = df['MES_ATUAL'].astype(float)  - df['MES_4'].astype(float)

##SEPARA POR MARCA
marca = df['Placa'].str.split(' ',n=1,expand=True)[0]
placa = df['Placa'].str.split(' ',n=1,expand=True)[1]
df['Marca'] = marca
df['Placa'] = placa

##corrigi classificação intel
df['Marca'].replace('Intel(R)', 'INTEL',inplace=True)
df['Marca'].replace('Other', 'OUTRAS', inplace=True)


df

,Placa,MES_1,MES_2,MES_3,MES_4,MES_ATUAL,RESULTADO,Marca
0,GeForce GTX 1650,6.19,6.07,5.67,5.30,5.11,-0.19,NVIDIA
1,GeForce RTX 3060,4.66,4.90,4.74,4.88,4.87,-0.01,NVIDIA
2,GeForce GTX 1060,4.99,4.94,4.58,4.60,4.31,-0.29,NVIDIA
3,GeForce RTX 3060 Laptop GPU,4.51,4.54,4.28,3.76,4.29,0.53,NVIDIA
4,GeForce RTX 2060,4.45,4.53,4.15,4.10,3.94,-0.16,NVIDIA
...,...,...,...,...,...,...,...,...
95,Radeon R7 Graphics,0.17,0.16,0.16,0.17,0.16,-0.01,AMD
96,GeForce GTX 750,0.19,0.17,0.18,0.17,0.16,-0.01,NVIDIA
97,HD Graphics 530,0,0,0.15,0.17,0.16,-0.01,Intel
98,None,9.44,9.48,9.92,10.01,10.06,0.05,OUTRAS


In [8]:
##ADICIONA TABELA DO MES
cursor.execute('alter table tb_placas_de_video_resultado add  AGOSTO NUMERIC(20,2)')
cursor.commit()

In [9]:
##CRIA TABELA TEMPORARIA PARA ARMAZENAR O RESULTADO DO  MES
cursor.execute('drop table if exists tb_placas_de_video_resultado_temp create table Tb_Placas_De_Video_Resultado_temp (placa varchar(200), MES_ATUAL NUMERIC(20,2),RESULTADO_MES NUMERIC(20,2))')
cursor.commit()




In [10]:
##insere na tabela os dados do mês
for index,row in df.iterrows():
    cursor.execute(f'insert into tb_placas_de_video_resultado_temp(placa,MES_ATUAL,RESULTADO_MES) values(?,?,?)',row.Placa,row.MES_ATUAL,row.RESULTADO)

cursor.commit()

In [11]:
##atualiza tabela principal
cursor.execute("""update tb_placas_de_video_resultado 
                  set AGOSTO = MES_ATUAL, RESULTADO = RESULTADO_MES
                  FROM tb_placas_de_video_resultado a
                  inner join tb_placas_de_video_resultado_temp b  on a.placa = b.placa   """)
cursor.commit()

#PARTE 3 PROCESSADOR


In [18]:
# Faz o download do conteúdo da página
url = "https://store.steampowered.com/hwsurvey/processormfg/"
response = requests.get(url)
content = response.content

# Cria um objeto BeautifulSoup para analisar o conteúdo HTML
soup = bs4.BeautifulSoup(content, "html.parser")

data_list = []
for i in range(0,2,1) :
# Encontra todas as divs com a classe "substats_row row_0"
    divs = soup.find_all("div", class_=f"substats_row row_{i}")

    # Extrai os dados de cada div
    for div in divs:
        data = div.get_text(separator='|', strip=True)
        data_list.append(data)

# transforma em dataframe
df = pd.DataFrame(data_list,columns=['A'])

##splita
df = df['A'].str.split('|',expand=True)

##renomeia as colunas 
df.columns = ['Marca', 'MES_1','MES_2','MES_3','MES_4','MES_ATUAL', 'RESULTADO']

##LIMPA CARACETER %
substituicoes = {'%':'','-':'0'}



df.iloc[0:,1:].replace(substituicoes,regex=True,inplace=True)

df.drop(columns='RESULTADO')

df['RESULTADO'] = df['MES_ATUAL'].astype(float)  - df['MES_4'].astype(float)

df

##filtra para achar somente as linhas que contem as marcas amd e intel
df = df.query("Marca == 'GenuineIntel' or Marca == 'AuthenticAMD' or Marca == 'MicrosoftXTA'").reset_index()

#seleciona somente o total por processador
df.drop(2,inplace=True)
df = df.iloc[0:3]

df = df.replace('GenuineIntel', 'INTEL')
df = df.replace('AuthenticAMD', 'AMD')
df = df.replace('MicrosoftXTA', 'Outros')

df

,index,Marca,MES_1,MES_2,MES_3,MES_4,MES_ATUAL,RESULTADO
0,0,INTEL,67.14,67.52,67.94,66.76,66.52,-0.24
1,1,Outros,0.03,0.03,0.03,0.05,0.05,0.00
3,29,AMD,32.84,32.46,32.03,33.19,33.43,0.24


In [44]:
##cria tabela para as informações de processadores
cursor = conn.cursor()
cursor.execute('drop table if exists tb_processadores  create table tb_processadores (MARCA varchar(200), MAR NUMERIC(20,2),ABRIL NUMERIC(20,2),MAIO NUMERIC(20,2),JUNHO NUMERIC(20,2),JULHO NUMERIC(20,2),RESULTADO NUMERIC(20,2))')
cursor.commit()


##insere dados na tabela
for index,row in df.iterrows():
    cursor.execute(f'insert into tb_processadores  (MARCA,MAR,ABRIL,MAIO,JUNHO,JULHO,RESULTADO) values(?,?,?,?,?,?,?)',row.Marca,row.MAR,row.ABRIL,row.MAIO,row.JUNHO,row.JULHO,row.RESULTADO)

cursor.commit()

In [23]:
#ATUALIZACAO MENSAL 

##ADICIONA COLUNA  DO MES
cursor.execute('alter table tb_processadores add  AGOSTO NUMERIC(20,2)')
cursor.commit()

##CRIA TABELA TEMPORARIA PARA ARMAZENAR O RESULTADO DO  MES
cursor.execute('drop table if exists tb_processadores_temp create table tb_processadores_temp (MARCA varchar(200), MES_ATUAL NUMERIC(20,2),RESULTADO_MES NUMERIC(20,2))')
cursor.commit()

##insere na tabela os dados do mês
for index,row in df.iterrows():
    cursor.execute(f'insert into tb_processadores_temp(MARCA,MES_ATUAL,RESULTADO_MES) values(?,?,?)',row.Marca,row.MES_ATUAL,row.RESULTADO)
cursor.commit()


##atualiza tabela principal -- lembrar de mudar o nome do mês
cursor.execute("""update tb_processadores
                  set AGOSTO = MES_ATUAL, 
                      RESULTADO = RESULTADO_MES
                  FROM tb_processadores a
                  inner join tb_processadores_temp b  on a.marca = b.marca   """)
cursor.commit()

In [ ]:
##top jogos mais vendidos

import requests
from bs4 import BeautifulSoup

def extrair_top_100_jogos():
    # URL da página com os top 100 jogos da Steam
    url = 'https://store.steampowered.com/search/?filter=topsellers'

    # Fazendo a requisição HTTP
    response = requests.get(url)

    # Verificando se a requisição foi bem-sucedida
    if response.status_code == 200:
        # Criando o objeto BeautifulSoup
        soup = BeautifulSoup(response.content, 'html.parser')

        # Encontrando a div que contém a lista de jogos
        games_div = soup.find('div', {'id': 'search_resultsRows'})

        # Iterando sobre os elementos da lista de jogos
        for game in games_div.find_all('a', {'class': 'search_result_row'}):
            # Extraindo o nome do jogo
            game_name = game.find('span', {'class': 'title'}).text.strip()
            print(game_name)

extrair_top_100_jogos()
